# Setup 

## Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import proplot as plot

sys.path.append('/Users/46h/Research/code/accphys') 
from tools import beam_analysis as ba
from tools import plotting as myplt
from tools.plotting import save, set_labels
from tools import animation as myanim
from tools import utils
from tools.utils import show, play, file_exists
from tools.accphys_utils import get_phase_adv
from tools.plot_utils import moment_label, moment_label_string

## Settings

In [ ]:
# Plotting
plt_kws = dict(legend=False)
plot.rc['figure.facecolor'] = 'white'
plot.rc['grid.alpha'] = 0.04
plot.rc['style'] = None 
plot.rc['savefig.dpi'] = 'figure' 
plot.rc['animation.html'] = 'jshtml'
dpi = 500

cmap = plot.Colormap('viridis')
cmap_range = (0, 1)
dpi = 500

## Read data

Load the files.

In [ ]:
tracked_params_list = np.load('_output/data/tracked_params_list.npy')
transfer_mats = np.load('_output/data/transfer_mats.npy')
angles = np.load('_output/data/angles.npy')
positions = np.load('_output/data/positions.npy')
positions_normed = positions / positions[-1]
mode = int(np.loadtxt('_output/data/mode.txt'))

Compute the beam statistics.

In [ ]:
stats_list = []

for tracked_params in tracked_params_list:
    
    stats = ba.Stats(mode)
    stats.read_env(tracked_params)
    stats.twiss2D['mux'] = get_phase_adv(stats.twiss2D['bx'], positions, units='deg')
    stats.twiss2D['muy'] = get_phase_adv(stats.twiss2D['by'], positions, units='deg')
    
    # Add position column
    for df in stats.dfs():
        df['s'] = positions
        df['s/L'] = positions_normed
    
    stats_list.append(stats)

Set color cycle from color map.

In [ ]:
colors = [cmap(i) for i in np.linspace(cmap_range[0], cmap_range[1], len(stats_list))]
_cycler = myplt.cycler('color', colors)

# Within lattice evolution
Show the s-dependent evolution of the beam parameters, with one curve for each value of the beam perveance. The lower end of the colormap corresponds to zero space charge.

## Twiss

### 2D Twiss 

In [ ]:
fig, axes = plot.subplots(ncols=2, nrows=3, figsize=(5, 5), spany=False, aligny=True)
axes.set_prop_cycle(_cycler)
cols_list = [['bx','by'], ['ax','ay'], ['ex','ey']]
for stats in stats_list:
    for ax, col in zip(axes, [col for cols in cols_list for col in cols]):
        stats.twiss2D[['s/L', col]].plot('s/L', ax=ax, **plt_kws)
ylabels = [r'$\beta$ [m]', r'$\alpha$ [rad]', r'$\varepsilon$ [mm $\cdot$ mrad]']
set_labels(axes[:, 0], ylabels, 'ylabel')
axes.format(xlabel='s / L', collabels=['Horizontal', 'Vertical'])
plt.savefig('_output/figures/twiss2D.png', dpi=dpi, facecolor='w')

### 4D Twiss

In [ ]:
fig, axes = plot.subplots(ncols=2, nrows=3, figsize=(5, 5), spany=False, aligny=True)
axes.set_prop_cycle(_cycler)
cols_list = [['bx','by'], ['ax','ay'], ['u']]
for stats in stats_list:
    for ax, col in zip(axes, [col for cols in cols_list for col in cols]):
        stats.twiss4D[['s/L', col]].plot('s/L', ax=ax, **plt_kws)
set_labels(axes[:, 0], [r'$\beta$ [m]', r'$\alpha$ [rad]', 'u'], 'ylabel')
axes.format(xlabel='s / L', collabels=['Horizontal', 'Vertical'])
plt.savefig('_output/figures/twiss4D.png', dpi=dpi, facecolor='w')

### Phase advance 

In [ ]:
fig, axes = plot.subplots(ncols=2, figsize=(6, 2.5))
axes.set_prop_cycle(_cycler)
for stats in stats_list:
    for ax, col in zip(axes, ['mux', 'muy']):
        stats.twiss2D[['s/L', col]].plot('s/L', ax=ax, **plt_kws)
axes.format(xlabel='s / L', ylabel='Phase advance', yformatter='deg',
            collabels=['Horizontal', 'Vertical'])
plt.savefig('_output/figures/phaseadv.png', dpi=dpi, facecolor='w')

### Phase difference (nu)

In [ ]:
fig, ax = plot.subplots(figsize=(3.5, 2.5))
ax.set_prop_cycle(_cycler)
for stats in stats_list:
    stats.twiss4D[['s/L','nu']].plot('s/L', ax=ax, **plt_kws)
ax.format(xlabel='s / L', ylabel=r'$\nu$', yformatter='deg')
plt.savefig('_output/figures/nu.png', dpi=dpi, facecolor='w')

## Moments 

In [ ]:
fig, axes = plot.subplots(ncols=2, nrows=3, figsize=(5, 5), spany=False, aligny=True)
axes.set_prop_cycle(_cycler)
cols_list = [['x_rms','y_rms'], ['xp_rms','yp_rms']]
for stats in stats_list:
    for ax, col in zip(axes, [col for cols in cols_list for col in cols]):
        stats.moments[['s/L', col]].plot('s/L', ax=ax, **plt_kws)
    stats.corr[['s/L','xy']].plot('s/L', ax=axes[2, 0], **plt_kws)
ylabels = ['Beam size [mm]', 'Beam div [mrad]', 'x-y corr. coef.']
set_labels(axes[:, 0], ylabels, 'ylabel')
axes.format(xlabel='s / L', collabels=['Horizontal', 'Vertical'])
plt.savefig('_output/figures/moments.png', dpi=dpi, facecolor='w')

In [ ]:
fig, axes = plot.subplots(nrows=4, ncols=4, sharey=False, figsize=(8, 6), 
                          spany=False, aligny=True)
myplt.make_lower_triangular(axes)
axes.set_prop_cycle(_cycler)
axes.format(xlabel='s / L', suptitle='Transverse moments', titleborder=True)
for stats in stats_list:
    for i in range(4):
        for j in range(i + 1):
            ax = axes[i, j]
            col = moment_label(i, j)
            stats.moments[['s/L', col]].plot('s/L', ax=ax, **plt_kws)
            ax.format(title=moment_label_string(i, j))
set_labels(axes[0:, 0], [r'[mm$^2$]', r'[mm$\cdot$mrad]', r'[mm$^2$]', r'mm$\cdot$mrad'], 'ylabel')
set_labels(axes[1:, 1], [r'[mrad$^2$]', r'[mm$\cdot$mrad]', r'[mrad$^2$]'], 'ylabel')
set_labels(axes[2:, 2], [r'[mm$^2$]', r'[mm$\cdot$mrad]'], 'ylabel')
set_labels(axes[3:, 3], [r'[mrad$^2$]'], 'ylabel')
plt.savefig('_output/figures/all_moments.png', dpi=dpi, facecolor='w')

In [ ]:
fig, axes = plot.subplots(nrows=4, ncols=4, sharey=False, figsize=(8, 6), 
                          spany=False, aligny=True)
myplt.make_lower_triangular(axes)
axes.set_prop_cycle(_cycler)
axes.format(xlabel='s / L', suptitle='Transverse moments', titleborder=True)
for stats in stats_list:
    for i in range(4):
        for j in range(i + 1):
            ax = axes[i, j]
            col = moment_label(i, j)
            stats.corr[['s/L', col]].plot('s/L', ax=ax, **plt_kws)
            ax.format(title=moment_label_string(i, j))
plt.savefig('_output/figures/all_correlations.png', dpi=dpi, facecolor='w')

## Real space orientation 

In [ ]:
# Choose 'area' for true area in x-y space, or 'area_rel' for fractional change
area = 'area_rel'

In [ ]:
if area == 'area':
    area_ylabel = r'Area [mm$^2$]' 
elif area == 'area_rel':
    area_ylabel = r'Area (frac. change)' 

fig, axes = plot.subplots(ncols=1, nrows=3, sharey=False, figsize=(3.5, 5))
axes.set_prop_cycle(_cycler)
for stats in stats_list:
    stats.realspace[['s/L','angle']].plot('s/L', ax=axes[0], **plt_kws)
    stats.realspace[['s/L', area]].plot('s/L', ax=axes[1], **plt_kws)
    stats.corr[['s/L','xy']].plot('s/L', ax=axes[2], **plt_kws)
axes.format(xlabel='s / L')
ylabels = ['Tilt angle [deg]', area_ylabel, 'x-y corr. coef.']
set_labels(axes, ylabels, 'ylabel')
plt.savefig('_output/figures/real_space_orientation.png', dpi=dpi, facecolor='w')

## Phase space projections 

In [ ]:
anim = myanim.corner_env(tracked_params_list, figsize=6, skip=9,
                         text_vals=positions_normed, text_fmt='s / L = {:.2f}',
                         cmap=cmap, cmap_range=cmap_range)
anim

# Fixed position in lattice

In [ ]:
position_to_observe = 0.0 # in units s/L
ind_var_name = 'quad tilt angle'
ind_var_vals = angles
discrete_kws = dict(lw=0, marker='o', legend=False)

Collect the parameters at this position.

In [ ]:
i = 0
for index, position_normed in enumerate(positions_normed):
    if position_normed >= position_to_observe:
        i = index
        s = position_normed
        break
print('Closest index is i = {}, with s/L = {:.3f}'.format(i, s))

In [ ]:
fixedpt = dict()
fixedpt['env_params'] = tracked_params_list[:, i, :]
fixedpt['twiss2D'] = pd.DataFrame(
    np.array([stats.twiss2D.iloc[i].values for stats in stats_list]),
    columns=stats.twiss2D.columns
)
fixedpt['twiss4D'] = pd.DataFrame(
    np.array([stats.twiss4D.iloc[i].values for stats in stats_list]),
    columns=stats.twiss4D.columns
)
fixedpt['moments'] = pd.DataFrame(
    np.array([stats.moments.iloc[i].values for stats in stats_list]),
    columns=stats.moments.columns
)
fixedpt['corr'] = pd.DataFrame(
    np.array([stats.corr.iloc[i].values for stats in stats_list]),
    columns=stats.corr.columns
)
fixedpt['realspace'] = pd.DataFrame(
    np.array([stats.realspace.iloc[i].values for stats in stats_list]),
    columns=stats.realspace.columns
)
for key, df in fixedpt.items():
    if key != 'env_params':
        df[ind_var_name] = ind_var_vals
    
fixedpt['twiss4D']

## Twiss 

### 2D Twiss 
Blue and orange correspond to $x$ and $y$, respectively.

In [ ]:
fig, axes = plot.subplots(nrows=3, figsize=(3, 5.5), sharey=False, aligny=True)
fixedpt['twiss2D'][[ind_var_name,'bx','by']].plot(ind_var_name, ax=axes[0], **discrete_kws)
fixedpt['twiss2D'][[ind_var_name,'ax','ay']].plot(ind_var_name, ax=axes[1], **discrete_kws)
fixedpt['twiss2D'][[ind_var_name,'ex', 'ey']].plot(ind_var_name, ax=axes[2], **discrete_kws)
axes.format(xformatter='deg')
axes[0].format(title='s / L = {:.2f}'.format(s))
axes[1].format(ylim=(-0.1, 0.1))
ylabels = [r'$\beta$ [m]', r'$\alpha$ [rad]', r'$\varepsilon$ [mm $\cdot$ mrad]']
set_labels(axes, ylabels, 'ylabel')
plt.savefig('_output/figures/twiss2D_fixedpt.png', dpi=dpi, facecolor='w')

### 4D Twiss 

In [ ]:
fig, axes = plot.subplots(nrows=3, figsize=(3, 5.5), sharey=False, aligny=True)
fixedpt['twiss4D'][[ind_var_name,'bx','by']].plot(ind_var_name, ax=axes[0], **discrete_kws)
fixedpt['twiss4D'][[ind_var_name,'ax','ay']].plot(ind_var_name, ax=axes[1], **discrete_kws)
fixedpt['twiss4D'][[ind_var_name,'u']].plot(ind_var_name, ax=axes[2], c='k', **discrete_kws)
axes[0].format(title='s / L = {:.2f}'.format(s))
axes.format(xformatter='deg')
axes[1].format(ylim=(-0.1, 0.1))
set_labels(axes, [r'$\beta$ [m]', r'$\alpha$ [rad]', 'u'], 'ylabel')
plt.savefig('_output/figures/twiss4D_fixedpt.png', dpi=dpi, facecolor='w')

### Phase advance 

In [ ]:
fig, ax = plot.subplots(figsize=(3, 2.5))
fixedpt['twiss2D'][[ind_var_name,'mux','muy']].plot(ind_var_name, ax=ax, **discrete_kws)
ax.format(title='s / L = {:.2f}'.format(s), xformatter='deg')
ax.format(xformatter='scalar', yformatter='deg', ylabel='Phase advance')
plt.savefig('_output/figures/phaseadv_fixedpt.png', dpi=dpi, facecolor='w')

### Phase difference (nu)

In [ ]:
fig, ax = plot.subplots(figsize=(3, 2.5))
fixedpt['twiss4D'][[ind_var_name,'nu']].plot(ind_var_name, ax=ax, color='k', **discrete_kws)
ax.format(title='s / L = {:.2f}'.format(s), xformatter='scalar')
ax.format(yformatter='deg', xformatter='deg', ylabel=r'$\nu$')
plt.savefig('_output/figures/nu_fixedpt.png', dpi=dpi, facecolor='w')

## Moments 

In [ ]:
fig, axes = plot.subplots(nrows=3, figsize=(3, 5.5), sharey=False, aligny=True)
fixedpt['moments'][[ind_var_name,'x_rms','y_rms']].plot(ind_var_name, ax=axes[0], **discrete_kws)
fixedpt['moments'][[ind_var_name,'xp_rms','yp_rms']].plot(ind_var_name, ax=axes[1], **discrete_kws)
fixedpt['corr'][[ind_var_name,'xy']].plot(ind_var_name, ax=axes[2], color='k', **discrete_kws)
axes[0].format(title='s / L = {:.2f}'.format(s))
axes.format(xformatter='deg')
ylabels = ['Beam size [mm]', 'Beam div. [mrad]', r'$x$-$y$ corr. coef.']
set_labels(axes, ylabels, 'ylabel')
plt.savefig('_output/figures/moments_fixedpt.png', dpi=dpi, facecolor='w')

## Real space orientation

In [ ]:
fig, axes = plot.subplots(nrows=3, figsize=(3, 5.5), sharey=False, aligny=True)
fixedpt['realspace'][[ind_var_name,'angle']].plot(ind_var_name, ax=axes[0], **discrete_kws)
fixedpt['realspace'][[ind_var_name,'cx','cy']].plot(ind_var_name, ax=axes[1], **discrete_kws)
fixedpt['realspace'][[ind_var_name,'area']].plot(ind_var_name, ax=axes[2], color='k', **discrete_kws)
set_labels(axes, ['tilt angle', 'ellipse axes [mm]', 'area [mm$^2$]'], 'ylabel')
axes.format(xformatter='deg')
plt.savefig('_output/figures/realspace_orientation_fixedpt.png', dpi=dpi, facecolor='w')

## Phase space projections

In [ ]:
axes = myplt.corner_env(fixedpt['env_params'], figsize=5, cmap=cmap, pad=0.25, lw=1)
plt.savefig('_output/figures/corner.png', dpi=dpi, facecolor='w')

In [ ]:
env_coords_fixedpt = [ba.get_ellipse_coords(p) for p in fixedpt['env_params']]

fig, axes = plt.subplots(1, 3, figsize=(7, 2), tight_layout=True)
myplt.despine(axes)
myplt.set_share_axes(axes[:2], sharex=True, sharey=True)
for ax in axes:
    ax.set_prop_cycle(_cycler)

for X in env_coords_fixedpt:
    x, xp, y, yp = X.T
    axes[0].plot(x, xp)
    axes[1].plot(y, yp)
for ind_var_val, u in zip(ind_var_vals, fixedpt['twiss4D'].loc[:, 'u']):
    axes[2].plot(ind_var_val, u, 'o')
    
set_labels(axes, ["x [mm]", "y [mm]", ind_var_name + ' [deg]'], 'xlabel')
set_labels(axes, ["x' [mrad]", "y' [mrad]"], 'ylabel')
set_labels(axes, ['Horiz. phase space', 'Vert. phase space', 'u'], 'title')
plt.savefig('_output/figures/horz_vert_phasespace.png', dpi=dpi, facecolor='w')

# Effective transfer matrix 

In [ ]:
max_number_of_columns = 8

In [ ]:
n = len(transfer_mats)
ncols = n if n <= max_number_of_columns else max_number_of_columns
nrows = int(np.ceil(n / max_number_of_columns))

## Eigenvalues 

In [ ]:
fig, axes = plot.subplots(ncols=ncols, nrows=nrows, figsize=(15, nrows*2.25))
axes.format(grid=False, xlim=(-1.3, 1.3), ylim=(-1.3, 1.3), xlabel='Real', ylabel='Imag')

psi = np.linspace(0, 2*np.pi, 50)
x_circle, y_circle = np.cos(psi), np.sin(psi)

for ax, M, ind_var_val in zip(axes, transfer_mats, ind_var_vals):
    ax.plot(x_circle, y_circle, 'k--', zorder=0)
    if not np.any(np.isnan(M)):
        eigvals = np.linalg.eigvals(M)
        eigtunes = np.degrees(np.arccos(eigvals.real))
        ax.scatter(eigvals.real, eigvals.imag, c=['r','r','b','b'], zorder=2)
        ax.annotate(r'$\mu_1 = {:.2f}\degree$'.format(eigtunes[0]), xy=(0.5, 0.55), xycoords='axes fraction', horizontalalignment='center')
        ax.annotate(r'$\mu_2 = {:.2f}\degree$'.format(eigtunes[2]), xy=(0.5, 0.45), xycoords='axes fraction', horizontalalignment='center')
    ax.format(title=r'{} = ${:.2f}\degree$'.format(ind_var_name, ind_var_val))
    ax.invert_xaxis() # ?

plt.savefig('_output/figures/transfer_mat_eigvals.png', dpi=dpi, facecolor='w')

## Eigenvectors 

In [ ]:
fig, axes = plot.subplots(ncols=ncols, nrows=nrows, figsize=(15, nrows*2.25))
axes.format(xlim=(-1.3, 1.3), ylim=(-1.3, 1.3), xlabel='x', ylabel='y')

psi = np.linspace(0, 2*np.pi, 50)
x_circle, y_circle = np.cos(psi), np.sin(psi)

for ax, M, ind_var_val in zip(axes, transfer_mats, ind_var_vals):
    myplt.eigvec_trajectory(ax, M, 'x', 'y', s=10, lw=2)
    ax.format(title=r'{} = ${:.2f}\degree$'.format(ind_var_name, ind_var_val))

plt.savefig('_output/figures/transfer_mat_eigvals.png', dpi=dpi, facecolor='w')